In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [50]:
train = pd.read_csv('train.csv', header=None)
test = pd.read_csv('test.csv', header=None)
train_label = pd.read_csv('trainLabels.csv', header=None)

In [51]:
train

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.299403,-1.226624,1.498425,-1.176150,5.289853,0.208297,2.404498,1.594506,-0.051608,0.663234,...,-0.850465,-0.622990,-1.833057,0.293024,3.552681,0.717611,3.305972,-2.715559,-2.682409,0.101050
1,-1.174176,0.332157,0.949919,-1.285328,2.199061,-0.151268,-0.427039,2.619246,-0.765884,-0.093780,...,-0.819750,0.012037,2.038836,0.468579,-0.517657,0.422326,0.803699,1.213219,1.382932,-1.817761
2,1.192222,-0.414371,0.067054,-2.233568,3.658881,0.089007,0.203439,-4.219054,-1.184919,-1.240310,...,-0.604501,0.750054,-3.360521,0.856988,-2.751451,-1.582735,1.672246,0.656438,-0.932473,2.987436
3,1.573270,-0.580318,-0.866332,-0.603812,3.125716,0.870321,-0.161992,4.499666,1.038741,-1.092716,...,1.022959,1.275598,-3.480110,-1.065252,2.153133,1.563539,2.767117,0.215748,0.619645,1.883397
4,-0.613071,-0.644204,1.112558,-0.032397,3.490142,-0.011935,1.443521,-4.290282,-1.761308,0.807652,...,0.513906,-1.803473,0.518579,-0.205029,-4.744566,-1.520015,1.830651,0.870772,-1.894609,0.408332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.310429,0.826811,-0.952245,0.768850,1.877520,1.320646,1.944609,1.191420,-0.127724,0.070937,...,-0.600411,-0.383792,0.745596,-0.698598,-2.729937,-0.431535,0.372873,1.019092,-2.672811,-0.295141
996,-1.853879,0.246726,0.459921,-2.074267,7.599220,-0.138355,-4.501900,0.630634,-1.590533,-1.112949,...,0.361736,0.240052,-0.856196,-0.072481,-2.935896,0.582411,-2.613407,0.036687,2.809310,4.412567
997,0.912748,-1.734039,-1.047035,0.217573,13.457812,0.162771,-2.250521,2.216161,-0.378326,0.642114,...,1.195896,-1.073806,-2.754369,1.814864,-4.190105,-1.116441,-2.100125,0.061513,0.895536,0.813686
998,2.439780,-0.735511,-0.902426,1.365036,-10.430299,-0.856859,2.686474,0.292035,0.585388,-0.876965,...,2.262326,-0.039488,0.773876,-0.916066,2.604827,-0.649874,-3.423674,0.229748,-2.311088,-3.422217


In [52]:
X_train = np.asarray(train)
Y_train = np.asarray(train_label).ravel()  # 展平为一维数组
X_test = np.asarray(test)

In [53]:
X_all = np.r_[X_train,X_test] #连接两个矩阵，列数相等
X_all

array([[ 0.29940251, -1.22662419,  1.49842505, ..., -2.71555881,
        -2.68240859,  0.10105047],
       [-1.17417585,  0.33215734,  0.94991875, ...,  1.21321926,
         1.38293163, -1.81776106],
       [ 1.19222208, -0.41437073,  0.06705418, ...,  0.6564375 ,
        -0.93247282,  2.9874358 ],
       ...,
       [ 0.05227421, -1.73655765, -0.26369872, ...,  1.1796059 ,
         1.15633997, -1.21856135],
       [ 1.44365875,  0.65189194,  0.55072377, ..., -0.42495429,
         1.33337425,  2.32527112],
       [-0.42914491, -0.11082132,  1.2572304 , ...,  0.61748452,
         1.46443756,  3.30144546]])

特征提取:寻找最佳的GMM模型并进行拟合和预测

In [54]:
from sklearn.mixture import GaussianMixture #高斯聚类

In [55]:
cv_types = ['spherical', 'tied', 'diag', 'full']
bic = []
lowest_bic = np.inf
best_gmm = None
n_components_range = range(1, 10) 

from sklearn.mixture import GaussianMixture

for cv_type in cv_types:
    for n_components in n_components_range:
        gmm = GaussianMixture(n_components=n_components, covariance_type=cv_type)
        gmm.fit(X_all)
        bic.append(gmm.bic(X_all))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

# 使用最佳模型进行预测
best_gmm.fit(X_all)
print("Best Gaussian Mixture Model:\n{}".format(best_gmm))

Best Gaussian Mixture Model:
GaussianMixture(n_components=4)


In [56]:
X_train = best_gmm.predict_proba(X_train)
X_test = best_gmm.predict_proba(X_test)

模型：随机森林

In [57]:
rf = RandomForestClassifier()

In [58]:
param_grid = dict( )
grid_search_rf = GridSearchCV(rf, param_grid=dict( ), verbose=3,scoring='accuracy',cv=10).fit(X_train,Y_train)
print('best estimator RandomForest:',grid_search_rf.best_estimator_,'Best Score', grid_search_rf.best_estimator_.score(X_train,Y_train))
rf_best = grid_search_rf.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 1/10] END .................................., score=1.000 total time=   0.1s
[CV 2/10] END .................................., score=1.000 total time=   0.1s
[CV 3/10] END .................................., score=1.000 total time=   0.1s
[CV 4/10] END .................................., score=0.990 total time=   0.1s
[CV 5/10] END .................................., score=1.000 total time=   0.1s
[CV 6/10] END .................................., score=1.000 total time=   0.1s
[CV 7/10] END .................................., score=0.990 total time=   0.1s
[CV 8/10] END .................................., score=1.000 total time=   0.1s
[CV 9/10] END .................................., score=0.980 total time=   0.1s
[CV 10/10] END ................................., score=1.000 total time=   0.1s
best estimator RandomForest: RandomForestClassifier() Best Score 0.998


In [59]:
classifier = RandomForestClassifier(
    n_estimators=1000,
    criterion='entropy',
    max_depth=5,
    min_samples_leaf=3,
    max_features='sqrt',
    bootstrap=False,
    oob_score=False,
    n_jobs=1,
    random_state=33,
    verbose=0
)

In [60]:
classifier.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5,
                       min_samples_leaf=3, n_estimators=1000, n_jobs=1,
                       random_state=33)

保存

In [62]:
rf_best_pred = pd.DataFrame(rf_best.predict(X_test))

In [63]:
rf_best_pred.index += 1
rf_best_pred.columns = ['Solution']
rf_best_pred['Id'] = np.arange(1,rf_best_pred.shape[0]+1)
rf_best_pred = rf_best_pred[['Id', 'Solution']]
print(rf_best_pred)

        Id  Solution
1        1         1
2        2         0
3        3         1
4        4         0
5        5         0
...    ...       ...
8996  8996         1
8997  8997         1
8998  8998         1
8999  8999         0
9000  9000         1

[9000 rows x 2 columns]


In [64]:
rf_best_pred.to_csv('Submission_rf.csv', index=False)